# Perguntas e respostas com o Bedrock

> *Este caderno deve funcionar bem com o kernel **`Data Science 3.0`** no SageMaker Studio*

Responder a perguntas (QA) é uma tarefa importante que envolve extrair respostas para consultas factuais feitas em uma linguagem natural. Normalmente, um sistema de QA processa uma consulta em uma base de conhecimento que contém dados estruturados ou não estruturados e gera uma resposta com informações precisas. Garantir um alto nível de acurácia é essencial para desenvolver um sistema de resposta a perguntas útil e confiável, principalmente para casos de uso empresarial. 

Modelos de IA generativa, como Titan e Claude, usam distribuições de probabilidade para gerar respostas para as perguntas. Esses modelos são treinados em vastas quantidades de dados de texto, o que permite que prevejam o que virá a seguir em uma sequência ou que palavra virá a seguir de outra. Entretanto, esses modelos não conseguem oferecer respostas precisas ou determinísticas para todas as perguntas porque sempre há um grau de incerteza nos dados. Empresas precisam consultar dados específicos de domínio e proprietários e usar as informações para responder a perguntas, além de dados mais genéricos para os quais o modelo não foi treinado. 

Neste módulo, demonstraremos como usar o modelo Bedrock Titan para fornecer informações para respostas a consultas.

Neste exemplo, executaremos o modelo sem contexto e, em seguida, tentaremos fornecer o contexto manualmente. Não acontece nenhum aumento por `RAG` aqui. Essa abordagem funciona para documentos curtos ou aplicações singleton e pode não se dimensionar para o nível empresarial de respostas a perguntas, no qual pode haver grandes documentos empresariais que podem não se encaixar no prompt enviado para o modelo. 

### Desafios
- Como fazer com o que o modelo dê uma resposta factual para a pergunta

### Proposta
Para os desafios acima, este caderno propõe a seguinte estratégia
#### Preparar documentos
Antes de poder responder às perguntas, os documentos devem ser processados e armazenados em um índice de armazenamento de documentos
- Aqui enviaremos a solicitação com o contexto relevante completo para o modelo e esperaremos a resposta


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import json
import os
import sys

import boto3
import botocore


module_path = ".."
sys.path.append(os.path.abspath(module_path))
from labutils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

## Seção 1: Perguntas e respostas com o conhecimento do modelo
Nesta seção, tentaremos usar modelos oferecidos pelo serviço do Bedrock para responder a perguntas com base no conhecimento que eles ganharam durante a fase de treinamento.

Neste caderno, utilizaremos o método `invoke_model()` do cliente Amazon Bedrock. Os parâmetros obrigatórios necessários para esse método são `modelId`, que representam o ARN do modelo do Amazon Bedrock, e `body`, que é o prompt da nossa tarefa. O prompt de `body` muda conforme o provedor do modelo de base selecionado. Veremos isso detalhadamente abaixo

```
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```


## Cenário

Estamos tentando criar um modelo para uma situação em que solicitamos que ele forneça informações para trocar pneus. Primeiro, pediremos que o modelo baseado nos dados de treinamento nos dê uma resposta para o modelo e marca específicos do nosso carro. Essa técnica é chamada de `Zero shot`. Perceberemos logo que, mesmo que pareça que o modelo está dando respostas que parecem relevantes para nosso carro específico, na verdade, ele está alucinando. É possível concluir isso porque executamos o processo com um carro falso e recebemos um cenários e respostas quase semelhantes

Essa situação indica que precisamos aumentar o treinamento do modelo com dados adicionais sobre a marca e modelo específicos do nosso carro e assim o modelo nos dará uma resposta muito específica. Neste caderno, não usaremos fontes externas para aumentar os dados, e sim simularemos com um sistema de aumento baseado em RAG funcionaria. 

Para realizar nosso teste final, fornecemos uma seção detalhada do nosso manual que explica como funciona a troca de pneus para nosso carro específico e testaremos como receber uma resposta com curadoria do modelo

## Tarefa

Para iniciar o processo, selecione um dos modelos fornecidos pelo Bedrock. Para esse caso de uso, selecione Titan. Esses modelos podem responder perguntas genéricas sobre carros.

Por exemplo, você pede que o modelo Titan diga como trocar um pneu furado do seu Audi.


In [ ]:
prompt_data = """You are an helpful assistant. Answer questions in a concise way. If you are unsure about the
answer say 'I am unsure'

Question: How can I fix a flat tire on my Audi A8?
Answer:"""
parameters = {
    "maxTokenCount":512,
    "stopSequences":[],
    "temperature":0,
    "topP":0.9
    }

#### Vamos invocar o modelo que passa pelo corpo JSON para gerar a resposta

In [ ]:
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-tg1-large"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"
try:
    
    response = boto3_bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())
    answer = response_body.get("results")[0].get("outputText")
    print_ww(answer.strip())

except botocore.exceptions.ClientError as error:
    if  error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Message']}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

O modelo consegue oferecer uma resposta descrevendo o processo de troca de um pneu furado do carro, mas a mesma explicação poderia ser válida para qualquer carro. Infelizmente, essa não é a resposta correta para um Audi A8, que não tem estepe. Isso acontece porque o modelo foi treinado com dados que contêm instruções sobre troca de pneus em carros.

Outro exemplo desse problema pode ser encontrado ao tentar fazer a mesma pergunta usando uma marca e modelo de carro totalmente falsos, como Amazon Tirana.

In [ ]:
prompt_data = "How can I fix a flat tire on my Amazon Tirana?"
body = json.dumps({"inputText": prompt_data, 
                   "textGenerationConfig": parameters})
modelId = "amazon.titan-tg1-large"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print_ww(answer.strip())

Como você pode ver, a resposta que o modelo oferece é plausível, mas é para uma bicicleta, não um carro. O modelo deduziu que Amazon Tirana é uma bicicleta. O modelo está _hallucinating_.

Como podemos corrigir esse problema e fazer com que o modelo dê respostas baseadas em instruções específicas e válidas para o modelo do meu carro?

Uma pesquisa feita pelo Facebook em 2020 revelou que o conhecimento de LLM poderia ser aumentado durante o processo com o fornecimento da base de conhecimento adicional como parte do prompt. Essa abordagem é chamada de geração aumentada de recuperação (Retrieval Augmented Generation, RAG).

Vamos ver como podemos usar isso para melhorar nossa aplicação.

A seguir, temos um trecho do manual do Audi A8 (na verdade, não se trata do manual real, mas vamos considerá-lo como tal). Este documento também é convenientemente curto o suficiente para caber por completo no prompt do Titan grande. 

```
Pneus e pressão dos pneus:

Os pneus são feitos de borracha preta e são montados nas rodas do seu veículo. Eles oferecem a aderência necessária para dirigir, manobrar e frear. Dois fatores importantes a considerar são a pressão e o desgaste dos pneus, pois eles podem afetar o desempenho e dirigibilidade do carro.

Onde encontrar a pressão recomendada para os pneus:

Você pode encontrar as especificações de pressão recomendada para o pneu na etiqueta de calibragem localizada na coluna B do lado do motorista no veículo. Como alternativa, você pode consultar o manual do veículo para encontrar esta informação. A pressão recomendada para o pneu pode variar dependendo da velocidade e do número de ocupantes ou carga máxima dentro do veículo.

Recalibragem dos pneus:

É importante realizar a verificação da pressão dos pneus quando eles estiverem frios. Isso significa deixar o veículo parado por pelo menos três horas para garantir que os pneus se igualem à temperatura ambiente.

Para recalibrar os pneus:

    Verifique a pressão recomendada para os pneus do seu veículo.
    Siga as instruções fornecidas na bomba pneumática e calibre um ou mais pneus com a pressão correta.
    No painel central do veículo, abra a aplicação “Status do carro”.
    Navegue até a aba “Pressão dos pneus”.
    Pressione a opção “Calibrar pressão” e confirme a ação.
    Dirija o carro por alguns minutos a uma velocidade acima de 30 km/h para calibrar a pressão dos pneus.

Observação: em alguns casos, pode ser necessário dirigir por mais de 15 minutos para apagar qualquer símbolo ou mensagem de aviso relacionado à pressão do pneu. Se os avisos persistirem, deixe os pneus esfriarem e repita as etapas acima.

Pneu furado:

Caso você descubra um pneu furado enquanto dirige, é possível selar temporariamente o furo e recalibrar o pneu usando um kit de mobilidade de pneu. Normalmente, esse kit fica armazenado no acabamento do porta-malas do veículo.

Instruções para utilizar o kit de mobilidade de pneu:

    Abra a tampa traseira ou o porta-malas do veículo.
    Levante o acabamento do porta-malas para acessar o kit de mobilidade de pneu.
    Siga as instruções fornecidas no kit de mobilidade de pneu para selar o furo do pneu.
    Após usar o kit, certifique-se de colocá-lo de volta no lugar original de forma segura.
    Entre em contato com a Rivesla ou com um serviço apropriado para receber ajuda no descarte e substituição da garrafa de selante usado.

Observe que o kit de mobilidade de pneu é uma solução temporária e foi projetado para permitir que você dirija por no máximo 10 minutos ou 8 km (o que acontecer primeiro) a uma velocidade máxima de 80 km/h. Aconselhamos que você troque o pneu furado ou encontre um profissional para repará-lo o mais rápido possível.
```

 
Em seguida, pegamos esse texto e o “incorporamos” ao prompt juntamente com a pergunta original. O prompt também foi criado de uma forma para tentar indicar que o modelo deve procurar somente nas informações fornecidas como contexto.

In [ ]:
context = """Tires and tire pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to find recommended tire pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for using the tire mobility kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact Audi or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible."""

#### Vamos pegar todo o trecho e colocá-lo no modelo juntamente com a pergunta.

In [ ]:
question = "How can I fix a flat tire on my Audi A8?"
prompt_data = f"""Answer the question based only on the information provided between ## and give step by step guide.
#
{context}
#

Question: {question}
Answer:"""

##### Invoque o modelo pelo boto3 para gerar a resposta

In [ ]:
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-tg1-large"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print_ww(answer.strip())

Como o modelo demora um pouco para entender o contexto e gerar respostas relevantes para você, isso pode resultar em uma experiência ruim para o usuário, já que ele pode ter que esperar alguns segundos por uma resposta.

O Bedrock também é compatível com transmissões em que o serviço gera uma saída conforme o modelo gera tokens. Segue um exemplo de como você pode fazer isso.

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

## Resumo

Podemos ver que a resposta é uma instrução resumida e com passo a passo de como trocar pneus. Este exemplo simples mostra como é possível aproveitar o `RAG` ou o processo de aumento para gerar uma resposta com curadoria